# Precursor
This was run on Google's Colab, which is why the training files and testing files are being loaded from the Drive.

Code below is to be able to access files in Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Base Model
- CNN with two 32 filter convolution layers and two 64 filter convolution layers
- Training data can be augmented to try and increase the sample size as a small sample size is used as training
- Some of the results of the are written in the comments at the top

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation, BatchNormalization
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from keras.regularizers import l1, l2

###############
## WITH DATA AUG
# 0 rotate, 0.1 hori vert shift, 0 zoom:
# loss: 0.6865 - acc: 0.7812 - val_loss: 0.4656 - val_acc: 0.8684 # USED THIS
# 30 rotate, 0.2 hori vert shift, 0.1 zoom:
# loss: 0.7520 - acc: 0.7368 - val_loss: 0.6760 - val_acc: 0.7961
## WITHOUT DATA AUG
# 0 rotate, 0.1 hori vert shift, 0 zoom:
# loss: 0.1997 - acc: 0.9457 - val_loss: 1.0672 - val_acc: 0.7171
# 30 rotate, 0.2 hori vert shift, 0.1 zoom:
# loss: 0.0930 - acc: 0.9737 - val_loss: 1.0791 - val_acc: 0.7566
###############

X_train = np.load(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/train_x.npy")
Y_train = np.load(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/train_label.npy")
X_test = np.load(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/test_x.npy")

NUM_CLASSES = 20
augment = True
EPOCHS = 50
BATCH_SIZE = 32

# Normalize
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes=NUM_CLASSES)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42)

# Set the CNN model
model = Sequential()
model.add(Conv2D(34, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

# Define the optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if augment:
  # Augment the data
  datagen = ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
      samplewise_center=False,  # set each sample mean to 0
      featurewise_std_normalization=False,  # divide inputs by std of the dataset
      samplewise_std_normalization=False,  # divide each input by its std
      zca_whitening=False,  # apply ZCA whitening
      zca_epsilon=1e-06,  # epsilon for ZCA whitening
      # randomly rotate images in the range (degrees, 0 to 180)
      rotation_range=0,
      # randomly shift images horizontally (fraction of total width)
      width_shift_range=0.1,
      # randomly shift images vertically (fraction of total height)
      height_shift_range=0.1,
      shear_range=0.,  # set range for random shear
      zoom_range=0.,  # set range for random zoom
      channel_shift_range=0.,  # set range for random channel shifts
      # set mode for filling points outside the input boundaries
      fill_mode='nearest',
      cval=0.,  # value used for fill_mode = "constant"
      horizontal_flip=True,  # randomly flip images
      vertical_flip=False,  # randomly flip images
      # set rescaling factor (applied before any other transformation)
      rescale=None,
      # set function that will be applied on each input
      preprocessing_function=None,
      # image data format, either "channels_first" or "channels_last"
      data_format=None,
      # fraction of images reserved for validation (strictly between 0 and 1)
      validation_split=0.0)

  # Compute quantities required for feature-wise normalization
  # (std, mean, and principal components if ZCA whitening is applied).
  datagen.fit(X_train)

  # Fit the model on the batches generated by datagen.flow().
  model.fit_generator(datagen.flow(X_train, Y_train,
                                   batch_size=BATCH_SIZE),
                      epochs=EPOCHS,
                      steps_per_epoch=np.ceil(len(X_train) / BATCH_SIZE),
                      validation_data=(X_val, Y_val),
                      workers=4)
else:
  model.fit(X_train, Y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(X_val, Y_val),
            shuffle=True)

results = model.predict(X_test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name='Predicted')

submission = pd.concat([pd.Series(range(0, 200), name='Id'), results], axis=1)

submission.to_csv(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/cnn_predicted.csv", index=False)


# ResNet Model
- CNN with using Keras' pretrained ResNet50 Model
- Training data is not augmented because it lowered the val_acc and increased the val_loss

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.utils.np_utils import to_categorical
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

X_train = np.load(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/train_x.npy")
Y_train = np.load(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/train_label.npy")
X_test = np.load(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/test_x.npy")

NUM_CLASSES = 20
EPOCHS = 10
BATCH_SIZE = 32


# Normalize
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes=NUM_CLASSES)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42)

INPUT_SHAPE = X_train.shape[1:]

# Set the CNN model - Using ResNet
base = ResNet50(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
pred = Dense(NUM_CLASSES, activation="softmax")(x)
model = Model(inputs=base.input, output=pred)

# Define the optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


# reduce the learning rate by half when the val_acc plateaus
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# early stop when val_loss doesnt decrease past baseline
early_stop = EarlyStopping(monitor='val_loss',
                           verbose=1,
                           mode='min',
                           baseline=0.05,
                           patience=5)

model.fit(X_train, Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(X_val, Y_val),
          shuffle=True,
          callbacks=[learning_rate_reduction, early_stop])

# Get results and write to csv
results = model.predict(X_test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name='Predicted')

submission = pd.concat([pd.Series(range(0, 200), name='Id'), results], axis=1)

submission.to_csv(
    "drive/My Drive/Colab Notebooks/syde 522 Kaggle Challenge/cnn_predicted.csv", index=False)
